In [ ]:
from bs4 import BeautifulSoup
import requests
from requests import get

import pandas as pd
import string
import re
import time

string.punctuation = string.punctuation + '•'

def cleaning_punct(text):
    for punc in string.punctuation:
        text = text.replace(punc, "")
    return text

def remove_extras(the_list, val):
   return [value for value in the_list if value != val]

In [236]:
# Links to Products Pages
links = ['https://www.astrotools.com/product/3-8-reversible-air-drill-1-800rpm/',
        'https://www.astrotools.com/product/1-2-extra-heavy-duty-reversible-air-drill-500rpm/',
        'https://www.astrotools.com/product/industrial-1-4-air-die-grinder/',
        'https://www.astrotools.com/product/1-2-air-ratchet-wrench-50ft-lb-torque/',
        'https://www.astrotools.com/product/1-2-super-duty-impact-wrench-twin-hammer/',
        'https://www.astrotools.com/product/1-heavy-duty-air-impact-wrench-with-2-anvil/',
        'https://www.astrotools.com/product/1-heavy-duty-air-impact-wrench-with-6-anvil/',
        'https://www.astrotools.com/product/onyx-heavy-duty-long-barrel-air-hammer-with-4pc-chisels-250mm/',
        'https://www.astrotools.com/product/needle-scaler-flux-hammer-combo-4-400-blows-per-minute/']

products_list = []

In [237]:
for link in links:
    url = requests.get(link)
    
    soup = BeautifulSoup(url.content, 'lxml')
    
    for elem in soup.find_all("br"):
        elem.replace_with(elem.text + "\n\n")
    
    #Product Title
    title_product = soup.find('h1').text

    #SKU
    sku = soup.find('span', class_='sku_wrapper').text.replace('SKU: ', '')
    
    #Category
    category = soup.find('span', class_='posted_in').text.replace('Categories: ', '')
    
    #Long Description
    title_description = soup.find('div', class_="et_pb_wc_description").text 

    title_desc = cleaning_punct(title_description)

    title_desc = title_desc.split('\n')

    desc = []
    for element in title_desc:
        el = element.strip()
        desc.append(el)

    desc = remove_extras(desc, '')

    description = ' '.join([str(elem) for elem in desc])
    
    #Air Inlet 
    product_spec = soup.find('div', class_="et_pb_tab clearfix").get_text().split('\n') 

    spec = []
    for element in product_spec:
        el = element.strip()
        spec.append(el)
    
    spec = remove_extras(spec, '')
    spec = remove_extras(spec, 'Specifications:')

    elements_inlet = []
    for element in spec:
       element1 = (re.match(r'^.*\b(Inlet)\b.*$', element))
       elements_inlet.append(element1)

    elements_inlet = remove_extras(elements_inlet, None)
    
    for element in elements_inlet:
        inlets = re.findall('\d+\W\d+', element.string)
        
        
    #Air Hose
    elements_hose = []
    for element in spec:
       element1 = (re.match(r'^.*\b(Hose|hose)\b.*$', element))
       elements_hose.append(element1)

    elements_hose = remove_extras(elements_hose, None)
  
    for element in elements_hose:
        hose = re.findall('\d+\W\d+', element.string)
       
        
    #Air Consumption
    elements_consumpt = []
    for element in spec:
       element1 = (re.match(r'^.*\b(Consumpt|consumption|Consumption)\b.*$', element))
       elements_consumpt.append(element1)

    elements_consumpt = remove_extras(elements_consumpt, None)
    
    for element in elements_consumpt:
        consumption = re.findall('[0-9]*\.?[0-9]+', element.string)
        
    #Free Speed
    elements_speed = []
    for element in spec:
       element1 = (re.match(r'^.*\b(Free|Blows)\b.*$', element))
       elements_speed.append(element1)

    elements_speed = remove_extras(elements_speed, None)
    
    for element in elements_speed:
        speed = re.findall('[0-9]*\,?[0-9]+', element.string)
        
    #Image Link
    link = soup.find_all('img')

    image_link = link[1].attrs['src']
    
    #Putting all together (title_product, sku, category, description, inlets, hose, consumption, pressure, speed, image_link)
    
    product = {
        'name': title_product,
        'SKU' : sku,
        'Category' : category,
        'Elaborate Description' : description,
        'Air Inlet Thread (inch)' : inlets[0],
        'Air Hose (inch)' : hose[0],
        'Avg Air Consumption (cfm)' : consumption[0],
        'Free Speed (bpm)' : speed[0],
        'Product Image' : image_link
    }
    
    time.sleep(30)
    
    products_list.append(product)
    
df = pd.DataFrame(products_list)
df

                                                name    SKU  \
0               3/8″ Reversible Air Drill – 1,800rpm   525C   
1  1/2″ Extra Heavy Duty Reversible Air Drill – 5...   527C   
2                    Industrial 1/4″ Air Die Grinder  210ES   
3         1/2″ Air Ratchet Wrench – 50ft./lb. Torque    131   
4        1/2″ Super Duty Impact Wrench – Twin Hammer   1812   
5      1″ Heavy Duty Air Impact Wrench with 2″ Anvil   1848   
6      1″ Heavy Duty Air Impact Wrench with 6″ Anvil   1849   
7  ONYX Heavy Duty Long Barrel Air Hammer with 4p...   4336   
8  Needle Scaler/Flux Hammer Combo – 4,400 blows ...   4350   

                           Category  \
0  Air Tools, Drills & Screwdrivers   
1  Air Tools, Drills & Screwdrivers   
2           Air Tools, Die Grinders   
3               Air Tools, Ratchets   
4        Air Tools, Impact Wrenches   
5           Category: Uncategorized   
6           Category: Uncategorized   
7              Air Tools, Specialty   
8              Air

In [238]:
df

,name,SKU,Category,Elaborate Description,Air Inlet Thread (inch),Air Hose (inch),Avg Air Consumption (cfm),Free Speed (bpm),Product Image
0,"3/8″ Reversible Air Drill – 1,800rpm",525C,"Air Tools, Drills & Screwdrivers",Lightweight compact size is easy to handle Pro...,1/4,3/8,7,"1,800",https://www.astrotools.com/wp-content/uploads/...
1,1/2″ Extra Heavy Duty Reversible Air Drill – 5...,527C,"Air Tools, Drills & Screwdrivers",Side handle for greater operator control One h...,1/4,3/8,7,500,https://www.astrotools.com/wp-content/uploads/...
2,Industrial 1/4″ Air Die Grinder,210ES,"Air Tools, Die Grinders",Allows light weld grinding porting breaking sh...,1/4,3/8,3,"20,000",https://www.astrotools.com/wp-content/uploads/...
3,1/2″ Air Ratchet Wrench – 50ft./lb. Torque,131,"Air Tools, Ratchets",For general automotive and industrial applicat...,1/4,3/8,6,150,https://www.astrotools.com/wp-content/uploads/...
4,1/2″ Super Duty Impact Wrench – Twin Hammer,1812,"Air Tools, Impact Wrenches",Ideal for general automotive work light truck ...,1/4,3/8,6.0,"7,500",https://www.astrotools.com/wp-content/uploads/...
5,1″ Heavy Duty Air Impact Wrench with 2″ Anvil,1848,Category: Uncategorized,NO LONGER AVAILABLE Ideal for tire work on tru...,1/4,3/4,11,"4,000",https://www.astrotools.com/wp-content/uploads/...
6,1″ Heavy Duty Air Impact Wrench with 6″ Anvil,1849,Category: Uncategorized,NO LONGER AVAILABLEIdeal for tire work on truc...,1/4,3/4,11,"4,000",https://www.astrotools.com/wp-content/uploads/...
7,ONYX Heavy Duty Long Barrel Air Hammer with 4p...,4336,"Air Tools, Specialty",NO LONGER AVAILABLEHandle Exhaust with Deflect...,1/4,3/8,3,2100,https://www.astrotools.com/wp-content/uploads/...
8,"Needle Scaler/Flux Hammer Combo – 4,400 blows ...",4350,"Air Tools, Specialty",Straight type air hammer and needle scaler Sui...,1/4,3/8,10,4400,https://www.astrotools.com/wp-content/uploads/...


In [250]:
astro_df = pd.read_csv(r'C:\Users\USER\Desktop\Penny\astro.csv', encoding="utf-8")
astro_df

,SKU,Module,Origin,Origin_Arabic,Bar_Code,Category,Arabic_Category,Procpal category,Sub-Category,Arabic_Sub_Category,...,Arabic_UoM,Weight(kg),Length(cm),Width(cm),Height(cm),Brand,Arabic_Brand,Source,Identifier,Unnamed: 27
0,ASTRO5225,5225,TAIWAN,تايوان,5225,Air / Pneumatic Tools,معدات تعمل بضغط الهواء,Air drills,Air Drills,دريل (هواء),...,حبة,1.10,24.0,17.0,5.3,Astro®,أسترو®,Astro Website,http://www.astrotools.com/air-tools/drills-scr...,NaN
1,ASTRO5275,5275,TAIWAN,تايوان,5275,Air / Pneumatic Tools,معدات تعمل بضغط الهواء,Air drills,Air Drills,دريل (هواء),...,حبة,1.80,22.5,17.0,5.5,Astro®,أسترو®,Astro Website,http://www.astrotools.com/1-2-extra-heavy-duty...,NaN
2,ASTRO1245,1245,TAIWAN,تايوان,7.45227E+11,Air / Pneumatic Tools,معدات تعمل بضغط الهواء,Air Impact Wrenches,Air Impact Wrenches,مفك صواميل (هواء),...,حبة,0.78,16.7,8.0,4.5,Astro®,أسترو®,Astro Website,http://www.astrotools.com/industrial-1-4-air-d...,NaN
3,ASTRO131,131AL,TAIWAN,تايوان,131AL,Air / Pneumatic Tools,معدات تعمل بضغط الهواء,Air Impact Wrenches,Air Impact Wrenches,مفك صواميل (هواء),...,حبة,1.30,27.0,8.0,4.5,Astro®,أسترو®,Astro Website,http://www.astrotools.com/1-2-air-ratchet-wren...,NaN
4,ASTRO1812,1812,TAIWAN,تايوان,7.45227E+11,Air / Pneumatic Tools,معدات تعمل بضغط الهواء,Air Impact Wrenches,Air Impact Wrenches,مفك صواميل (هواء),...,حبة,2.83,21.3,19.8,7.7,Astro®,أسترو®,Astro Website,http://www.astrotools.com/1-2-super-duty-impac...,NaN
5,ASTRO1868,1868,TAIWAN,تايوان,7.45227E+11,Air / Pneumatic Tools,معدات تعمل بضغط الهواء,Air Impact Wrenches,Air Impact Wrenches,مفك صواميل (هواء),...,حبة,11.60,43.0,25.0,15.8,Astro®,أسترو®,Astro Website,http://www.astrotools.com/1-heavy-duty-air-imp...,NaN
6,ASTRO1869,1869,TAIWAN,تايوان,7.45227E+11,Air / Pneumatic Tools,معدات تعمل بضغط الهواء,Air Impact Wrenches,Air Impact Wrenches,مفك صواميل (هواء),...,حبة,11.60,55.8,19.3,13.4,Astro®,أسترو®,Astro Website,http://www.astrotools.com/1-heavy-duty-air-imp...,NaN
7,ASTRO4336,4336,TAIWAN,تايوان,7.45227E+11,Air / Pneumatic Tools,معدات تعمل بضغط الهواء,Air Hammers,Air Hammers,مكينة تكسير (هواء),...,حبة,2.22,29.0,17.0,6.0,Astro®,أسترو®,Astro Website,http://www.astrotools.com/onyx-heavy-duty-long...,NaN
8,ASTRO4350,4350,TAIWAN,تايوان,7.45227E+11,Air / Pneumatic Tools,معدات تعمل بضغط الهواء,Air Hammers,Air Hammers,مكينة تكسير (هواء),...,حبة,3.16,30.3,13.5,5.4,Astro®,أسترو®,Astro Website,http://www.astrotools.com/needle-scaler-flux-h...,NaN


In [252]:
astro_df['Module'] = df['SKU']
astro_df['Category'] = df['Category']
astro_df['Air Inlet Thread (inch)'] = df['Air Inlet Thread (inch)']
astro_df['Air Hose (inch)'] = df['Air Hose (inch)']
astro_df['Avg Air Consumption (cfm)'] = df['Avg Air Consumption (cfm)']
astro_df['Free Speed (bpm)'] = df['Free Speed (bpm)']
astro_df['Product Image'] = df['Product Image']
astro_df['Elaborate Description'] = df['Elaborate Description']

astro_df


,SKU,Module,Origin,Origin_Arabic,Bar_Code,Category,Arabic_Category,Procpal category,Sub-Category,Arabic_Sub_Category,...,Source,Identifier,Unnamed: 27,module,Air Inlet Thread (inch),Air Hose (inch),Avg Air Consumption (cfm),Free Speed (bpm),Product Image,Elaborate Description
0,ASTRO5225,525C,TAIWAN,تايوان,5225,"Air Tools, Drills & Screwdrivers",معدات تعمل بضغط الهواء,Air drills,Air Drills,دريل (هواء),...,Astro Website,http://www.astrotools.com/air-tools/drills-scr...,NaN,525C,1/4,3/8,7,"1,800",https://www.astrotools.com/wp-content/uploads/...,Lightweight compact size is easy to handle Pro...
1,ASTRO5275,527C,TAIWAN,تايوان,5275,"Air Tools, Drills & Screwdrivers",معدات تعمل بضغط الهواء,Air drills,Air Drills,دريل (هواء),...,Astro Website,http://www.astrotools.com/1-2-extra-heavy-duty...,NaN,527C,1/4,3/8,7,500,https://www.astrotools.com/wp-content/uploads/...,Side handle for greater operator control One h...
2,ASTRO1245,210ES,TAIWAN,تايوان,7.45227E+11,"Air Tools, Die Grinders",معدات تعمل بضغط الهواء,Air Impact Wrenches,Air Impact Wrenches,مفك صواميل (هواء),...,Astro Website,http://www.astrotools.com/industrial-1-4-air-d...,NaN,210ES,1/4,3/8,3,"20,000",https://www.astrotools.com/wp-content/uploads/...,Allows light weld grinding porting breaking sh...
3,ASTRO131,131,TAIWAN,تايوان,131AL,"Air Tools, Ratchets",معدات تعمل بضغط الهواء,Air Impact Wrenches,Air Impact Wrenches,مفك صواميل (هواء),...,Astro Website,http://www.astrotools.com/1-2-air-ratchet-wren...,NaN,131,1/4,3/8,6,150,https://www.astrotools.com/wp-content/uploads/...,For general automotive and industrial applicat...
4,ASTRO1812,1812,TAIWAN,تايوان,7.45227E+11,"Air Tools, Impact Wrenches",معدات تعمل بضغط الهواء,Air Impact Wrenches,Air Impact Wrenches,مفك صواميل (هواء),...,Astro Website,http://www.astrotools.com/1-2-super-duty-impac...,NaN,1812,1/4,3/8,6.0,"7,500",https://www.astrotools.com/wp-content/uploads/...,Ideal for general automotive work light truck ...
5,ASTRO1868,1848,TAIWAN,تايوان,7.45227E+11,Category: Uncategorized,معدات تعمل بضغط الهواء,Air Impact Wrenches,Air Impact Wrenches,مفك صواميل (هواء),...,Astro Website,http://www.astrotools.com/1-heavy-duty-air-imp...,NaN,1848,1/4,3/4,11,"4,000",https://www.astrotools.com/wp-content/uploads/...,NO LONGER AVAILABLE Ideal for tire work on tru...
6,ASTRO1869,1849,TAIWAN,تايوان,7.45227E+11,Category: Uncategorized,معدات تعمل بضغط الهواء,Air Impact Wrenches,Air Impact Wrenches,مفك صواميل (هواء),...,Astro Website,http://www.astrotools.com/1-heavy-duty-air-imp...,NaN,1849,1/4,3/4,11,"4,000",https://www.astrotools.com/wp-content/uploads/...,NO LONGER AVAILABLEIdeal for tire work on truc...
7,ASTRO4336,4336,TAIWAN,تايوان,7.45227E+11,"Air Tools, Specialty",معدات تعمل بضغط الهواء,Air Hammers,Air Hammers,مكينة تكسير (هواء),...,Astro Website,http://www.astrotools.com/onyx-heavy-duty-long...,NaN,4336,1/4,3/8,3,2100,https://www.astrotools.com/wp-content/uploads/...,NO LONGER AVAILABLEHandle Exhaust with Deflect...
8,ASTRO4350,4350,TAIWAN,تايوان,7.45227E+11,"Air Tools, Specialty",معدات تعمل بضغط الهواء,Air Hammers,Air Hammers,مكينة تكسير (هواء),...,Astro Website,http://www.astrotools.com/needle-scaler-flux-h...,NaN,4350,1/4,3/8,10,4400,https://www.astrotools.com/wp-content/uploads/...,Straight type air hammer and needle scaler Sui...


In [259]:
astro_df.drop(columns = ['Unnamed: 27'], inplace = True)

astro_df

,SKU,Module,Origin,Origin_Arabic,Bar_Code,Category,Arabic_Category,Procpal category,Sub-Category,Arabic_Sub_Category,...,Brand,Arabic_Brand,Source,Identifier,Air Inlet Thread (inch),Air Hose (inch),Avg Air Consumption (cfm),Free Speed (bpm),Product Image,Elaborate Description
0,ASTRO5225,525C,TAIWAN,تايوان,5225,"Air Tools, Drills & Screwdrivers",معدات تعمل بضغط الهواء,Air drills,Air Drills,دريل (هواء),...,Astro®,أسترو®,Astro Website,http://www.astrotools.com/air-tools/drills-scr...,1/4,3/8,7,"1,800",https://www.astrotools.com/wp-content/uploads/...,Lightweight compact size is easy to handle Pro...
1,ASTRO5275,527C,TAIWAN,تايوان,5275,"Air Tools, Drills & Screwdrivers",معدات تعمل بضغط الهواء,Air drills,Air Drills,دريل (هواء),...,Astro®,أسترو®,Astro Website,http://www.astrotools.com/1-2-extra-heavy-duty...,1/4,3/8,7,500,https://www.astrotools.com/wp-content/uploads/...,Side handle for greater operator control One h...
2,ASTRO1245,210ES,TAIWAN,تايوان,7.45227E+11,"Air Tools, Die Grinders",معدات تعمل بضغط الهواء,Air Impact Wrenches,Air Impact Wrenches,مفك صواميل (هواء),...,Astro®,أسترو®,Astro Website,http://www.astrotools.com/industrial-1-4-air-d...,1/4,3/8,3,"20,000",https://www.astrotools.com/wp-content/uploads/...,Allows light weld grinding porting breaking sh...
3,ASTRO131,131,TAIWAN,تايوان,131AL,"Air Tools, Ratchets",معدات تعمل بضغط الهواء,Air Impact Wrenches,Air Impact Wrenches,مفك صواميل (هواء),...,Astro®,أسترو®,Astro Website,http://www.astrotools.com/1-2-air-ratchet-wren...,1/4,3/8,6,150,https://www.astrotools.com/wp-content/uploads/...,For general automotive and industrial applicat...
4,ASTRO1812,1812,TAIWAN,تايوان,7.45227E+11,"Air Tools, Impact Wrenches",معدات تعمل بضغط الهواء,Air Impact Wrenches,Air Impact Wrenches,مفك صواميل (هواء),...,Astro®,أسترو®,Astro Website,http://www.astrotools.com/1-2-super-duty-impac...,1/4,3/8,6.0,"7,500",https://www.astrotools.com/wp-content/uploads/...,Ideal for general automotive work light truck ...
5,ASTRO1868,1848,TAIWAN,تايوان,7.45227E+11,Category: Uncategorized,معدات تعمل بضغط الهواء,Air Impact Wrenches,Air Impact Wrenches,مفك صواميل (هواء),...,Astro®,أسترو®,Astro Website,http://www.astrotools.com/1-heavy-duty-air-imp...,1/4,3/4,11,"4,000",https://www.astrotools.com/wp-content/uploads/...,NO LONGER AVAILABLE Ideal for tire work on tru...
6,ASTRO1869,1849,TAIWAN,تايوان,7.45227E+11,Category: Uncategorized,معدات تعمل بضغط الهواء,Air Impact Wrenches,Air Impact Wrenches,مفك صواميل (هواء),...,Astro®,أسترو®,Astro Website,http://www.astrotools.com/1-heavy-duty-air-imp...,1/4,3/4,11,"4,000",https://www.astrotools.com/wp-content/uploads/...,NO LONGER AVAILABLEIdeal for tire work on truc...
7,ASTRO4336,4336,TAIWAN,تايوان,7.45227E+11,"Air Tools, Specialty",معدات تعمل بضغط الهواء,Air Hammers,Air Hammers,مكينة تكسير (هواء),...,Astro®,أسترو®,Astro Website,http://www.astrotools.com/onyx-heavy-duty-long...,1/4,3/8,3,2100,https://www.astrotools.com/wp-content/uploads/...,NO LONGER AVAILABLEHandle Exhaust with Deflect...
8,ASTRO4350,4350,TAIWAN,تايوان,7.45227E+11,"Air Tools, Specialty",معدات تعمل بضغط الهواء,Air Hammers,Air Hammers,مكينة تكسير (هواء),...,Astro®,أسترو®,Astro Website,http://www.astrotools.com/needle-scaler-flux-h...,1/4,3/8,10,4400,https://www.astrotools.com/wp-content/uploads/...,Straight type air hammer and needle scaler Sui...


In [266]:
astro_df.to_csv(r'C:\Users\USER\Desktop\Penny\astro_new.csv', index = False, encoding='utf-8-sig')